In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import rotate

Setting environment for Gordon


In [2]:
cell_masks_normalized = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/fractal_dim/cell_masks_normalized.bp')

In [3]:
n_cells = len(cell_masks_normalized)

In [6]:
from registration_utilities import *

In [5]:
nonsmall_cells_indices = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/fractal_dim/nonsmallCells_indices_cutoff30.bp')

In [8]:
n_nonsmallCells = len(nonsmall_cells_indices)

In [10]:
nonsmall_cell_masks_normalized = cell_masks_normalized[nonsmall_cells_indices].copy()

In [15]:
# E = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/fractal_dim/embeddingAllEigenNormalized_nonsmallCells_nystromSample100.bp')
E = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/fractal_dim/embeddingAllEigenNormalized_nonsmallCells_nystromSample200.bp')

In [ ]:
! rm -r /home/yuncong/csd395/CSHL_cells/gallery/nonsmallKMeans/*

In [17]:
n = 100

colors = random_colors(n)

indices = np.random.choice(np.arange(n_nonsmallCells), n, replace=False)

sizes = np.zeros((n, 2), np.int)

for r, i in enumerate(indices):
    
    contour_mask = np.zeros_like(nonsmall_cell_masks_normalized[i])
    contour_pts = np.array(find_contour_points(nonsmall_cell_masks_normalized[i].astype(np.int), sample_every=1)[1][0], np.int)
    contour_mask[contour_pts[:,1], contour_pts[:,0]] = 1
    xmin, xmax, ymin, ymax = bbox_2d(contour_mask)      
    m = contour_mask[ymin:ymax+1, xmin:xmax+1].copy()

    m_rgba = np.zeros(m.shape+(4,), np.uint8)
    m_rgba[m > 0, :3] = colors[r]
    m_rgba[m > 0, 3] = 255
    
    contour_output_dir = create_if_not_exists('/home/yuncong/csd395/CSHL_cells/gallery/nonsmallKMeans/random%d/contours' % (n))
    imsave(contour_output_dir + '/cellMask_random%dContours_%d_%d.png' % (n,r, i), m_rgba)
    
    xmin, xmax, ymin, ymax = bbox_2d(nonsmall_cell_masks_normalized[i])  
    m = nonsmall_cell_masks_normalized[i][ymin:ymax+1, xmin:xmax+1].copy()

    m_rgba = np.zeros(m.shape+(4,), np.uint8)
    m_rgba[m > 0, :3] = colors[r]
    m_rgba[m > 0, 3] = 255
    
    blob_output_dir = create_if_not_exists('/home/yuncong/csd395/CSHL_cells/gallery/nonsmallKMeans/random%d/blobs' % (n))
    imsave(blob_output_dir + '/cellMask_random%dBlobs_%d_%d.png' % (n, r, i), m_rgba)

    sizes[r] = m_rgba.shape[:2]
    
    
E_dict = []

for r, i in enumerate(indices):
    e1, e2, e3, e4, e5 = E[i, :5]
    s = sizes[r]
    E_dict.append({'e1':e1, 'e2':e2, 'e3':e3, 'e4':e4, 'e5':e5, 'id':r, 'index':i, 'w':s[1], 'h':s[0]})

import json
json.dump(E_dict, open('/home/yuncong/csd395/CSHL_cells/gallery/nonsmallKMeans/random%d/embedding_random%d.json' % (n, n), 'w'))

In [ ]:
! cd /home/yuncong/csd395/CSHL_cells/gallery/; tar -czf random.tar.gz random